In [ ]:
# Make sure you have a GPU running
!nvidia-smi

Tue Mar 16 13:06:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Install the latest release of Haystack in your own environment 
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install urllib3==1.25.4

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-5_wkasj3
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-5_wkasj3
     |████████████████████████████████| 215kB 21.8MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
     |████████████████████████████████| 81kB 12.9MB/s 
     |████████████████████████████████| 327kB 39.1MB/s 
     |████████████████████████████████| 327kB 45.6MB/s 
     |████████████████████████████████| 92kB 11.6MB/s 
     |████████████████████████████████| 983kB 54.3MB/s 
     |████████████████████████████████| 5.5MB 49.7MB/s 
     |████████████████████████████████| 143kB 55.7MB/s 
     |████████████████████████████████| 8.2MB 44.5MB/s 
     |████████████████████████████████| 225kB 55.2MB/s 
     |████████████████████████████████| 61kB 9.8MB/s 
     |████████████████████████████████| 3.0MB 47.1MB/s 
     |████████████████████████████████

In [ ]:
from haystack import Finder
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore

from haystack.retriever.dense import EmbeddingRetriever
from haystack.utils import print_answers
import pandas as pd
import requests


03/16/2021 13:09:24 - INFO - faiss.loader -   Loading faiss with AVX2 support.
03/16/2021 13:09:24 - INFO - faiss.loader -   Loading faiss.
03/16/2021 13:09:25 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [ ]:
# Recommended: Start Elasticsearch using Docker
# ! docker run -d -p 9200:9200 -e "discovery.type=single-node" elasticsearch:7.9.2

In [ ]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.10.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.10.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.10.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.10.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30


In [ ]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="",
                                            index="medicalfaq",
                                            embedding_field="question_emb",
                                            embedding_dim=768,
                                            excluded_meta_data=["question_emb"])



03/16/2021 13:10:09 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.072s]
03/16/2021 13:10:09 - INFO - elasticsearch -   PUT http://localhost:9200/text [status:200 request:0.335s]
03/16/2021 13:10:10 - INFO - elasticsearch -   PUT http://localhost:9200/label [status:200 request:0.192s]


In [ ]:
retriever = EmbeddingRetriever(document_store=document_store, embedding_model="deepset/sentence_bert", use_gpu=True)

03/16/2021 13:10:10 - INFO - haystack.retriever.dense -   Init retriever using embeddings of model deepset/sentence_bert
03/16/2021 13:10:10 - INFO - farm.utils -   Using device: CUDA 
03/16/2021 13:10:10 - INFO - farm.utils -   Number of GPUs: 1
03/16/2021 13:10:10 - INFO - farm.utils -   Distributed Training: False
03/16/2021 13:10:10 - INFO - farm.utils -   Automatic Mixed Precision: None
03/16/2021 13:10:10 - INFO - filelock -   Lock 139724718947280 acquired on /root/.cache/huggingface/transformers/f2fa4ddfd1da74d4f9fb00a8879d6730346d543bb6576db37fecddf4b2da4ae1.e9c7ea4b16d7c4a79864df631645d5aefec00f2a258f27810e0ae70be87de670.lock


03/16/2021 13:10:10 - INFO - filelock -   Lock 139724718947280 released on /root/.cache/huggingface/transformers/f2fa4ddfd1da74d4f9fb00a8879d6730346d543bb6576db37fecddf4b2da4ae1.e9c7ea4b16d7c4a79864df631645d5aefec00f2a258f27810e0ae70be87de670.lock
03/16/2021 13:10:10 - INFO - filelock -   Lock 139724409401936 acquired on /root/.cache/huggingface/transformers/2e610e46f4b28817d45cca7015637eee4b8d4ac6f0c77f2bdf43098cab361f9a.53dde806a91354b9d84223dcf593417ea6da6a3fb31a23e2e799621881366daf.lock


03/16/2021 13:10:27 - INFO - filelock -   Lock 139724409401936 released on /root/.cache/huggingface/transformers/2e610e46f4b28817d45cca7015637eee4b8d4ac6f0c77f2bdf43098cab361f9a.53dde806a91354b9d84223dcf593417ea6da6a3fb31a23e2e799621881366daf.lock


03/16/2021 13:10:33 - INFO - filelock -   Lock 139724395726352 acquired on /root/.cache/huggingface/transformers/e40ccd5a52ae8819bfadcaa8b736c78f3a1ebab84336a7701cf34a271933200a.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


03/16/2021 13:10:33 - INFO - filelock -   Lock 139724395726352 released on /root/.cache/huggingface/transformers/e40ccd5a52ae8819bfadcaa8b736c78f3a1ebab84336a7701cf34a271933200a.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
03/16/2021 13:10:33 - INFO - filelock -   Lock 139724395727120 acquired on /root/.cache/huggingface/transformers/94db90ea267e3fc9718b08857d0a301c4adf8df38df2b22d86876c2b5e1daef2.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b.lock


03/16/2021 13:10:33 - INFO - filelock -   Lock 139724395727120 released on /root/.cache/huggingface/transformers/94db90ea267e3fc9718b08857d0a301c4adf8df38df2b22d86876c2b5e1daef2.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b.lock
03/16/2021 13:10:33 - INFO - filelock -   Lock 139724391819664 acquired on /root/.cache/huggingface/transformers/058c0f259a3d40715957f88c3fb02faef02f13cfe69e33d6bd2f4ee07cf05def.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock


03/16/2021 13:10:33 - INFO - filelock -   Lock 139724391819664 released on /root/.cache/huggingface/transformers/058c0f259a3d40715957f88c3fb02faef02f13cfe69e33d6bd2f4ee07cf05def.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock


03/16/2021 13:10:33 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
03/16/2021 13:10:33 - INFO - farm.utils -   Using device: CUDA 
03/16/2021 13:10:33 - INFO - farm.utils -   Number of GPUs: 1
03/16/2021 13:10:33 - INFO - farm.utils -   Distributed Training: False
03/16/2021 13:10:33 - INFO - farm.utils -   Automatic Mixed Precision: None
03/16/2021 13:10:33 - WARNING - haystack.retriever.dense -   You seem to be using a Sentence Transformer with the dot_product function. We recommend using cosine instead. This can be set when initializing the DocumentStore


In [ ]:
# Get dataframe with columns "question", "answer" and some custom metadata
df = pd.read_csv("Corpus_New1.csv")
# Minimal cleaning
df.fillna(value="", inplace=True)
df["Question"] = df["Question"].apply(lambda x: x.strip())
print(df.head())

# Get embeddings for our questions from the FAQs
questions = list(df["Question"].values)
df["question_emb"] = retriever.embed_queries(texts=questions)
df = df.rename(columns={"Question": "text"})

# Convert Dataframe to list of dicts and index them in our DocumentStore
docs_to_index = df.to_dict(orient="records")
document_store.write_documents(docs_to_index)

In [ ]:
from haystack.pipeline import FAQPipeline
pipe = FAQPipeline(retriever=retriever)

In [ ]:
prediction = pipe.run(query="Is glaucoma caused in the cornea of the eye?", top_k_retriever=10)
ans = str(prediction)
print_answers(prediction, details="all")

with open( 'jsson.txt', encoding='utf-8', mode='w') as f2:
  f2.write(ans)


In [ ]:
wholejson = ""
wholejson = str(document_store.get_all_documents(return_embedding=False))
with open( 'wholejson.txt', encoding='utf-8', mode='w') as f3:
  f3.write(wholejson)

02/25/2021 17:36:52 - INFO - elasticsearch -   POST http://localhost:9200/document/_search?scroll=1d&size=10000 [status:200 request:0.108s]
02/25/2021 17:36:52 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.016s]
02/25/2021 17:36:52 - INFO - elasticsearch -   DELETE http://localhost:9200/_search/scroll [status:200 request:0.006s]
